### Imports

##### Ensure you've installed ffmpeg and added it to your path!

In [1]:
import gym
from gym import wrappers
import io
import base64
from IPython.display import HTML
import random
import numpy as np
import warnings
warnings.simplefilter('ignore')

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from collections import deque

#### Quick test to make sure everything works

In [5]:
# Create cartpole enviornment
env = gym.make('CartPole-v0')

# create wrapper to capture the results and store them
env = wrappers.Monitor(env, "./gym-results", force=True)
env.reset()
for _ in range(10000):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    if done: break
env.close()

In [6]:
# get the results of the simulation
video = io.open('./gym-results/openaigym.video.%s.video000000.mp4' % env.file_infix, 'r+b').read()
encoded = base64.b64encode(video)

#send the simulation video to notebook output
HTML(data='''
    <video width="360" height="auto" alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''
.format(encoded.decode('ascii')))

#### Quick test of DQNAgent

In [2]:
EPISODES = 10

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

#### 10 episode test (interupted)

In [4]:
if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    agent.load("./save_best/cartpole-dqn.h5")
    done = False
    batch_size = 32

    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(100):
            env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
        # if e % 10 == 0:
        #     agent.save("./save/cartpole-dqn.h5")

episode: 0/10, score: 15, e: 1.0
episode: 1/10, score: 25, e: 0.96
episode: 2/10, score: 56, e: 0.72


KeyboardInterrupt: 

In [5]:
env.close()

#### Longer episode test (interupted)

In [14]:
EPISODES = 1000


if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    # agent.load("./save/cartpole-dqn.h5")
    done = False
    batch_size = 32

    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            # env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)

episode: 0/1000, score: 17, e: 1.0
episode: 1/1000, score: 17, e: 0.99
episode: 2/1000, score: 16, e: 0.91
episode: 3/1000, score: 20, e: 0.82
episode: 4/1000, score: 12, e: 0.77
episode: 5/1000, score: 16, e: 0.71
episode: 6/1000, score: 9, e: 0.68
episode: 7/1000, score: 23, e: 0.61
episode: 8/1000, score: 23, e: 0.54
episode: 9/1000, score: 14, e: 0.51
episode: 10/1000, score: 18, e: 0.46
episode: 11/1000, score: 9, e: 0.44
episode: 12/1000, score: 14, e: 0.41
episode: 13/1000, score: 12, e: 0.39
episode: 14/1000, score: 9, e: 0.37
episode: 15/1000, score: 11, e: 0.35
episode: 16/1000, score: 10, e: 0.33
episode: 17/1000, score: 10, e: 0.32
episode: 18/1000, score: 10, e: 0.3
episode: 19/1000, score: 11, e: 0.29
episode: 20/1000, score: 10, e: 0.27
episode: 21/1000, score: 8, e: 0.26
episode: 22/1000, score: 14, e: 0.24
episode: 23/1000, score: 9, e: 0.23
episode: 24/1000, score: 9, e: 0.22
episode: 25/1000, score: 14, e: 0.21
episode: 26/1000, score: 9, e: 0.2
episode: 27/1000, sco

KeyboardInterrupt: 

In [5]:
env.close()